<a href="https://colab.research.google.com/github/scionrails/KIPRIS/blob/main/train_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Google Drive 경로 설정
import os

# -------------------------------
# 모델 저장 경로
# -------------------------------
MODEL_SAVE_DIR  = "/content/drive/MyDrive/Research/microstructual images of steels and properities/metal_model/best_models"
os.makedirs(MODEL_SAVE_DIR , exist_ok=True)
MODEL_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, f"best_vgg16_seed{SEED}.pth")
BEST_INFO_PATH = os.path.join(MODEL_SAVE_DIR, f"best_vgg16_seed{SEED}_info.json")

In [ ]:
from google.colab import files
import os, shutil

# 업로드 요청
print("🔐 kaggle.json 파일을 업로드하세요 (API Key)")
uploaded = files.upload()

# 파일명 자동 추출
for filename in uploaded.keys():
    if filename.endswith(".json"):
        kaggle_json_path = filename
        break
else:
    raise ValueError("❌ JSON 형식의 파일이 없습니다.")

# ~/.kaggle 폴더 생성
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# 복사 및 권한 설정
shutil.copy(kaggle_json_path, os.path.join(kaggle_dir, "kaggle.json"))

# 복사 후 원본 삭제
os.remove(kaggle_json_path)

os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)

print("✅ kaggle.json 설정 완료 (내용은 출력하지 않음)")


🔐 kaggle.json 파일을 업로드하세요 (API Key)


Saving kaggle.json to kaggle.json
✅ kaggle.json 설정 완료 (내용은 출력하지 않음)


In [ ]:
# 3. 데이터셋 다운로드 및 압축 해제
!kaggle datasets download -d safi842/highcarbon-micrographs --unzip -p ./metal_dataset

Dataset URL: https://www.kaggle.com/datasets/safi842/highcarbon-micrographs
License(s): copyright-authors
 91% 390M/426M [00:00<00:00, 501MB/s]
100% 426M/426M [00:00<00:00, 504MB/s]


In [ ]:
import os
import shutil
import pandas as pd

# 경로 설정
excel_path = "metal_dataset/new_metadata.xlsx"
image_dir = "metal_dataset/For Training/Cropped"  # 실제 경로 반영
output_dir = "metal_dataset_sorted"

# 엑셀 읽기
df = pd.read_excel(excel_path)

# 폴더 정리 시작
for _, row in df.iterrows():
    filename = row['path']  # 예: 'micrograph973.png'
    real_filename = f"Cropped{filename}"  # 실제 존재하는 파일 이름
    label = str(row['primary_microconstituent']).strip().lower().replace(" ", "_").replace("+", "_plus_")

    src = os.path.join(image_dir, real_filename)
    dst_dir = os.path.join(output_dir, label)
    dst = os.path.join(dst_dir, real_filename)

    os.makedirs(dst_dir, exist_ok=True)

    if os.path.exists(src):
        shutil.copy(src, dst)
    else:
        print(f"[경고] 이미지 파일 없음: {src}")


In [ ]:
# check_label_distribution.py

import pandas as pd

# 엑셀 경로
excel_path = "metal_dataset/new_metadata.xlsx"

# 라벨 정규화 함수 정의
def normalize_label(label):
    if pd.isna(label):
        return "unknown"
    return str(label).strip().lower().replace(" ", "_").replace("+", "_plus_")

# 엑셀 파일 읽기
df = pd.read_excel(excel_path)

# 라벨 정규화
df['normalized_label'] = df['primary_microconstituent'].apply(normalize_label)

# 고유 라벨 분포 집계
label_counts = df['normalized_label'].value_counts()

# 결과 출력
print(f"전체 이미지 수 (엑셀 기준): {len(df)}")
print("고유 라벨 분포:")
print(label_counts)

# 결과를 파일로 저장 (선택)
label_counts.to_csv("label_distribution_summary.csv")

전체 이미지 수 (엑셀 기준): 598
고유 라벨 분포:
normalized_label
spheroidite                       372
network                           101
spheroidite_plus_widmanstatten     77
pearlite_plus_spheroidite          28
pearlite                           15
pearlite_plus_widmanstatten         5
Name: count, dtype: int64


In [ ]:
# 03_split_dataset.py

import os
import shutil
import random

# 설정
INPUT_DIR = "metal_dataset_sorted"
OUTPUT_DIR = "dataset_split"
SPLIT_RATIOS = (0.8, 0.1, 0.1)  # train : val : test

# 시드 고정
random.seed(42)

# 클래스 순회
for class_name in os.listdir(INPUT_DIR):
    class_path = os.path.join(INPUT_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.lower().endswith((".png", ".jpg", ".jpeg"))]
    total = len(images)
    random.shuffle(images)

    # 기본 분할 수 계산
    n_train = int(total * SPLIT_RATIOS[0])
    n_val   = int(total * SPLIT_RATIOS[1])
    n_test  = total - n_train - n_val

    # 최소 1장씩 보장
    if total >= 3:
        n_train = max(n_train, 1)
        n_val = max(n_val, 1)
        n_test = max(n_test, 1)

        while n_train + n_val + n_test > total:
            n_train -= 1  # 총합이 초과되면 하나 줄임
    elif total == 2:
        n_train, n_val, n_test = 1, 1, 0
    elif total == 1:
        n_train, n_val, n_test = 1, 0, 0
    else:
        print(f"[주의] {class_name} 클래스에 이미지 없음. 건너뜀.")
        continue

    split_sets = {
        'train': images[:n_train],
        'val': images[n_train:n_train + n_val],
        'test': images[n_train + n_val:n_train + n_val + n_test]
    }

    # 디렉토리 생성 및 이미지 복사
    for split_name, split_images in split_sets.items():
        split_dir = os.path.join(OUTPUT_DIR, split_name, class_name)
        os.makedirs(split_dir, exist_ok=True)

        for img_name in split_images:
            src = os.path.join(class_path, img_name)
            dst = os.path.join(split_dir, img_name)
            shutil.copy2(src, dst)

    print(f"{class_name:<40} → 총 {total}장 | train: {len(split_sets['train'])}, val: {len(split_sets['val'])}, test: {len(split_sets['test'])}")

pearlite_plus_spheroidite                → 총 28장 | train: 22, val: 2, test: 4
network                                  → 총 101장 | train: 80, val: 10, test: 11
pearlite_plus_widmanstatten              → 총 5장 | train: 3, val: 1, test: 1
spheroidite                              → 총 372장 | train: 297, val: 37, test: 38
spheroidite_plus_widmanstatten           → 총 77장 | train: 61, val: 7, test: 9
pearlite                                 → 총 15장 | train: 12, val: 1, test: 2


In [ ]:
# train_vgg.py

import os
import json
import argparse
from collections import Counter
import random, numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms, models
from tqdm import tqdm

# -------------------------------
# 인자 설정
# -------------------------------
SEED = 42

# -------------------------------
# 시드 고정
# -------------------------------
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# -------------------------------
# 설정
# -------------------------------
DATASET_DIR = "dataset_split"
BATCH_SIZE = 32
NUM_WORKERS = 0
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 6
NUM_EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# -------------------------------
# Transform 정의
# -------------------------------
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


# VGG16 모델 정의
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# -------------------------------
# 데이터셋 로딩
# -------------------------------
train_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, "train"), transform=train_transform)
val_dataset   = datasets.ImageFolder(os.path.join(DATASET_DIR, "val"), transform=val_transform)

with open("class_to_idx.json", "w") as f:
    json.dump(train_dataset.class_to_idx, f, indent=2)

def get_sampler(dataset):
    targets = dataset.targets
    class_counts = Counter(targets)
    class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
    sample_weights = [class_weights[t] for t in targets]
    return WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=get_sampler(train_dataset), num_workers=NUM_WORKERS)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# -------------------------------
# 학습 루프
# -------------------------------
best_acc = 0.0
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}/{NUM_EPOCHS}]"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss /= total

    # 검증
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")

    # -------------------------------
    # 모델 저장
    # -------------------------------
    if val_acc > best_acc:
        best_acc = val_acc

        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch + 1,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'seed': SEED,
        }, MODEL_SAVE_PATH)

        best_info = {
            'model_path': MODEL_SAVE_PATH,
            'train_acc': round(train_acc, 4),
            'val_acc': round(val_acc, 4),
            'epoch': epoch + 1,
            'seed': SEED
        }

        with open(BEST_INFO_PATH, 'w') as f:
            json.dump(best_info, f, indent=2)

        print(f"✅ 모델 저장됨: {MODEL_SAVE_PATH}")
        print(f"ℹ️  Best model info 저장됨: {BEST_INFO_PATH}")

print(f"\n🎯 학습 완료. 최고 검증 정확도: {best_acc:.4f}")

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 160MB/s]
[Epoch 1/20]: 100%|██████████| 15/15 [00:14<00:00,  1.06it/s]


Epoch 1: Train Loss = 1.1372, Train Acc = 0.5726, Val Acc = 0.8276
✅ 모델 저장됨: /content/drive/MyDrive/Research/microstructual images of steels and properities/metal_model/best_models/best_vgg16_seed42.pth
ℹ️  Best model info 저장됨: /content/drive/MyDrive/Research/microstructual images of steels and properities/metal_model/best_models/best_vgg16_seed42_info.json


[Epoch 2/20]: 100%|██████████| 15/15 [00:13<00:00,  1.09it/s]


Epoch 2: Train Loss = 0.4413, Train Acc = 0.8316, Val Acc = 0.7931


[Epoch 3/20]: 100%|██████████| 15/15 [00:14<00:00,  1.01it/s]


...

Epoch 18: Train Loss = 0.0753, Train Acc = 0.9789, Val Acc = 0.8966


[Epoch 19/20]: 100%|██████████| 15/15 [00:13<00:00,  1.14it/s]


Epoch 19: Train Loss = 0.0509, Train Acc = 0.9832, Val Acc = 0.8621


[Epoch 20/20]: 100%|██████████| 15/15 [00:13<00:00,  1.14it/s]


Epoch 20: Train Loss = 0.0155, Train Acc = 0.9958, Val Acc = 0.8793

🎯 학습 완료. 최고 검증 정확도: 0.9138
